In [5]:
import torch.nn.functional as F
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

classes = [
    "Customer Service",
    "Audio",
    "Visual",
    "Bluetooth",
    "Microphone",
    "Battery Life",
    "Internet",
    "Performance/Speed",
    "Value/Price",
    "Ease of Use",
    "Comfortability/Fit",
    "Undetermined"
]
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for id, class_ in enumerate(classes)}
model_path = "microsoft/deberta-v3-large"

tokenizer = AutoTokenizer.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(classes), id2label=id2class, label2id=class2id, problem_type="multi_label_classification")
model.load_state_dict(torch.load('classification_beta/model_epoch_2.pth'))

/home/dyang/miniconda3/envs/NLP_env3.8/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [7]:
# custom input into the model
custom_text = "The shape of the mouse is very uncomfortable, it makes my hand hurt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenized_text = tokenizer(custom_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
tokenized_text = {k: v.to(device) for k, v in tokenized_text.items()}
output = model(**tokenized_text).to(device)

predicted_labels = torch.max(output.logits, 1)[1]
predicted_class = id2class[predicted_labels.item()]

print(f"Predicted class: {predicted_class}")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)